In [1]:
import pandas as pd
import numpy as np
import multiprocessing
from pandarallel import pandarallel

In [17]:
pandarallel.initialize(progress_bar=True,nb_workers=15,shm_size_mb=3500)


New pandarallel memory created - Size: 3500 MB
Pandarallel will run on 15 workers


In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
def convert2txt(path):
    d = []
    with open(path) as file:
        for i,line in enumerate(file):
                d.append(line)
    return d

In [5]:
a = convert2txt('./article_categories_en.ttl')

In [7]:
with open('./article_categories_en.txt', 'w') as f:
    for item in a:
        f.write("%s" % item)

In [8]:
d = pd.read_csv('./article_categories_en.txt', header=None, delimiter="\t")

In [9]:
d.head()

,0
0,# started 2015-11-06T08:48:44Z
1,<http://dbpedia.org/resource/Albedo> <http://p...
2,<http://dbpedia.org/resource/Albedo> <http://p...
3,<http://dbpedia.org/resource/Albedo> <http://p...
4,<http://dbpedia.org/resource/Albedo> <http://p...


In [36]:
d_ = d[0].str.split(' ', expand=True)

In [41]:
d1 = d_.iloc[1:-1]
d1 = pd.DataFrame(d1)
d1.drop(columns=[1],inplace=True)
d2 = d1.iloc[:,:2]

In [50]:
d1.head()

,0,2,3,4,5,6,7,8,9,10,11,12,13
1,<http://dbpedia.org/resource/Albedo>,<http://dbpedia.org/resource/Category:Climate_...,.,None,None,None,None,None,None,None,None,None,None
2,<http://dbpedia.org/resource/Albedo>,<http://dbpedia.org/resource/Category:Climatol...,.,None,None,None,None,None,None,None,None,None,None
3,<http://dbpedia.org/resource/Albedo>,<http://dbpedia.org/resource/Category:Electrom...,.,None,None,None,None,None,None,None,None,None,None
4,<http://dbpedia.org/resource/Albedo>,<http://dbpedia.org/resource/Category:Radiometry>,.,None,None,None,None,None,None,None,None,None,None
5,<http://dbpedia.org/resource/Albedo>,<http://dbpedia.org/resource/Category:Scatteri...,.,None,None,None,None,None,None,None,None,None,None


In [57]:
d2.head(10)

,0,2,0_en,2_cat
1,<http://dbpedia.org/resource/Albedo>,<http://dbpedia.org/resource/Category:Climate_...,Albedo,Climate_forcing
2,<http://dbpedia.org/resource/Albedo>,<http://dbpedia.org/resource/Category:Climatol...,Albedo,Climatology
3,<http://dbpedia.org/resource/Albedo>,<http://dbpedia.org/resource/Category:Electrom...,Albedo,Electromagnetic_radiation
4,<http://dbpedia.org/resource/Albedo>,<http://dbpedia.org/resource/Category:Radiometry>,Albedo,Radiometry
5,<http://dbpedia.org/resource/Albedo>,<http://dbpedia.org/resource/Category:Scatteri...,Albedo,"Scattering,_absorption_and_radiative_transfer_..."
6,<http://dbpedia.org/resource/Albedo>,<http://dbpedia.org/resource/Category:Radiation>,Albedo,Radiation
7,<http://dbpedia.org/resource/Anarchism>,<http://dbpedia.org/resource/Category:Anarchism>,Anarchism,Anarchism
8,<http://dbpedia.org/resource/Anarchism>,<http://dbpedia.org/resource/Category:Politica...,Anarchism,Political_culture
9,<http://dbpedia.org/resource/Anarchism>,<http://dbpedia.org/resource/Category:Politica...,Anarchism,Political_ideologies
10,<http://dbpedia.org/resource/Anarchism>,<http://dbpedia.org/resource/Category:Social_t...,Anarchism,Social_theories


In [53]:
d2['0_en'] = d2[0].apply(lambda x: x.split("/resource/")[-1][:-1])

/home/vibhav/bar/virtualenv/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [56]:
d2['2_cat'] = d2[2].apply(lambda x: x.split("Category:")[-1][:-1])

/home/vibhav/bar/virtualenv/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [62]:
d2.to_csv('./list_article_categories.csv',index=None)

In [71]:
d3 = d2.loc[:,['0_en','2_cat']]

In [75]:
d4 = d3.groupby('0_en')

In [80]:
d4.get_group('Albedo')

,2_cat
1,Climate_forcing
2,Climatology
3,Electromagnetic_radiation
4,Radiometry
5,"Scattering,_absorption_and_radiative_transfer_..."
6,Radiation


In [85]:
d5 = d4['2_cat'].apply('_/:/_'.join)

In [102]:
d5.head(10)

0_en
                        <http://purl.org/dc/terms/subject_/:/_<http://...
!!!                     Rock_music_groups_from_California_/:/_Musical_...
!!!_(album)             !!!_albums_/:/_2001_debut_albums_/:/_Gold_Stan...
!!Destroy-Oh-Boy!!      1993_debut_albums_/:/_New_Bomb_Turks_albums_/:...
!!M                                             Instant_messaging_clients
!Action_Pact!           Musical_groups_established_in_1981_/:/_Musical...
!Arriba!_La_Pachanga                    1961_albums_/:/_Latin_jazz_albums
!Hero                                                         Rock_operas
!Hero_(album)           2003_albums_/:/_Rock_operas_/:/_Contemporary_C...
!Kung_language          Kx'a_languages_/:/_Languages_of_Angola_/:/_Lan...
Name: 2_cat, dtype: object

In [121]:
d6 = pd.DataFrame(d5.iloc[1:])

In [122]:
d6.head()

,2_cat
0_en,
!!!,Rock_music_groups_from_California_/:/_Musical_...
!!!_(album),!!!_albums_/:/_2001_debut_albums_/:/_Gold_Stan...
!!Destroy-Oh-Boy!!,1993_debut_albums_/:/_New_Bomb_Turks_albums_/:...
!!M,Instant_messaging_clients
!Action_Pact!,Musical_groups_established_in_1981_/:/_Musical...


In [123]:
d6.to_csv('./list_article_categories_grouped.csv')

In [125]:
x1 = pd.read_csv('./list_article_categories_grouped.csv')

In [126]:
x1.columns

Index(['0_en', '2_cat'], dtype='object')